## First, use lmod to load fsl in a specific version

In [ ]:
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.5.1')
await lmod.list()

## Then run the pipeline for cortical and subcortical segmentation

In [ ]:
%%bash

first_run=
lm1="$(mktemp /tmp/lm1.$$.XXXX)"
lm2="$(mktemp /tmp/lm2.$$.XXXX)"
logf="final.log"
if [ -z "$FSLDIR" ]; then
    FSLDIR=/opt/fsl-6.0.5.1
fi

declare -a niifile=($(find "$PWD" -type f -iname "*.nii"));
for i in {1..2};
do
	for filename in "${niifile[@]}";
		do 
		 
  			echo ${filename}
			echo "running FIRST"
			image=$(basename ${filename%.*})
			run_first_all -i ${filename} -o ${filename%.*}
			declare -a files=(${filename%.*}.logs/*.e*);
			for f in ${files[@]};
			do
				if [ -s $f ]; then
					echo $f
					find . -type f -name ${image}* -exec rm -fv {} +
					#echo ${image}
					break
				fi	
			done	

		done

	for filename in "${niifile[@]}";
		do 
			echo ${filename}
			echo "running BET"
			bet ${filename}.nii ${filename%.*}_brain
			echo "running FAST"
			fast ${filename%.*}_brain.nii.gz	
		done
	
	find . -type f -name '*brain.nii.gz' -name 'brain_seg.nii.gz' -name '*all_fast_firstseg.nii.gz' -name '*to_std_sub.nii.gz' -exec md5sum {} > $lm1 \;
	if [ -z "$first_run" ]; then
        cp -a $lm1 $lm2
        first_run=1
    else
        first_run=2
    fi

    # test ! to invert usual exit code
    if ! diff $lm2 $lm1; then
        echo -e "$(date +"%F %R")\tChange detected:\t$1" | tee -a "$logf"
    else
        echo "same" & echo "$(md5sum $lm1)"
    fi	
    	
    mv $lm1 $lm2
    echo $first_run
	if [ $first_run == 1 ]; then
        find . ! -name '*.nii' ! -name '*.ipynb' -type f -exec rm -f {} +
    fi
done

for filename in "${niifile[@]}";
	do 
	echo ${filename}
	flirt -in  ${filename%.*}_all_fast_firstseg.nii.gz -ref ${FSLDIR}/data/standard/MNI152_T1_1mm.nii.gz -applyxfm -init  ${filename%.*}_to_std_sub.mat -out  ${filename%.*}_all_fast_firstseg_mni.nii.gz -interp nearestneighbour
	done

echo "All done"